In [ ]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
from pathlib import Path

In [ ]:
import ecephys_analyses as ea
from ecephys.sglx_utils import load_trigger
import ecephys.signal.timefrequency as tfr

In [ ]:
xr.set_options(keep_attrs=True)

In [ ]:
def get_bandpower(sig, chans, f_range):
    fft_window_length = 4 # seconds
    nperseg = int(fft_window_length * sig.fs) 
    noverlap = nperseg // 4 # 1 second overlap
    spg = tfr.parallel_spectrogram_welch(sig, nperseg=nperseg, noverlap=noverlap).median(dim="channel")
    return tfr.get_bandpower(spg, f_range)

In [8]:
# This should probably not compute cx power when sr chans is absent. 

def get_condition_bandpower(subject, experiment, condition, probe):
    cx_chans = ea.get_channels(subject, experiment, probe, "superficial_ctx", asarray=False)
    wm_chans = ea.get_channels(subject, experiment, probe, "white_matter", asarray=False)
    
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "sr_chans.csv")
    power_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "bandpower.nc")
    
    for bin_path, sr_chans_path, power_path in zip(bin_paths, sr_chans_paths, power_paths):
        sr_chans_df = ea.load_sr_chans(sr_chans_path)
        epochs = list()
        for epoch in sr_chans_df.itertuples():
            if not epoch.sr_chans:
                continue
            cx = load_trigger(bin_path, cx_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            wm = load_trigger(bin_path, wm_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            sr = load_trigger(bin_path, epoch.sr_chans, start_time=epoch.start_time, end_time=epoch.end_time)

            cx_wm_ref = cx - wm.values
            cx_wm_ref.attrs['reference'] = wm.channel.values

            sr_wm_ref = sr - wm.values
            sr_wm_ref.attrs['reference'] = wm.channel.values

            delta = (0.5, 4)
            theta = (5, 10)
            power = xr.Dataset({'mpta_delta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, delta),
                                'mpta_theta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, theta),
                                'sr_delta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, delta),
                                'sr_theta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, theta)})
            epochs.append(power)
            
        if not epochs:
            continue
        power = xr.concat(epochs, dim="time")
        Path(power_path).parent.mkdir(parents=True, exist_ok=True) # Create parent directories if they do not already exist.
        power.to_netcdf(power_path)
        power.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [9]:
get_condition_bandpower("Eugene", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
18:10:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs_g0/9.24.2020_BL_24hs_g0_imec0/9.24.2020_BL_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
18:13:38: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
18:17:02: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
18:20:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020

In [10]:
get_condition_bandpower("Allan", "sleep-homeostasis", "all", "imec1")

nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
19:25:21: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_g0/3-1-2021_g0_imec1/3-1-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 361262
nChan: 385, nFileSamp: 361262
nChan: 385, nFileSamp: 361262
19:25:30: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021-a_g0/3-1-2021-a_g0_imec1/3-1-2021-a_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 3863847
nChan: 385, nFileSamp: 3863847
nChan: 385, nFileSamp: 3863847
19:27:05: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g0/3-1-2021_A_g0_imec1/3-1-2021_A_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
19:30:32: Fin

In [11]:
get_condition_bandpower("Doppio", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
21:31:45: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
21:38:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
21:45:48: Finished /Volumes/neuropixel

In [12]:
get_condition_bandpower("Valentino", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
00:00:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
00:07:54: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
00:14:47: Finished /Volumes/neur

In [13]:
get_condition_bandpower("Segundo", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
02:53:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
03:00:27: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
03:07:08: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t10.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
03:13:47: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t11.imec0.lf.bin
nChan: 385, nFileSamp: 1800000